In [99]:
from rdflib import Graph, Namespace, URIRef
from typing import Optional
from pydantic import BaseModel
from api.src.function.loc.getType import GetType
from pyfuseki import FusekiUpdate
import httpx
from pysolr import Solr

In [10]:
au_update = FusekiUpdate('http://localhost:3030', 'authorities')

r = au_update.run_sparql('DROP DEFAULT')
r.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

In [2]:
from api.src.schemas.authorities.authority import Variant, AdminMetadata, Element
class Uri(BaseModel):
    value: str
    label: str
    base: Optional[str]

class Affiliation(BaseModel):
    organization: Uri
    affiliationStart: str
    affiliationEnd: Optional[str]
    
class Agents(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    fullerName: Optional[Element]
    birthDate: Optional[str]
    birthPlace: Optional[str]
    deathDate: Optional[str]
    hasAffiliation: Optional[list[Affiliation]]
    occupation: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    contributorOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str
    

In [55]:
authority = 'http://id.loc.gov/authorities/names/n80002329'
graph = Graph()
graph.parse(f'{authority}.rdf')
# graph.serialize('saramago.ttl')

<Graph identifier=N2efcebb891464f7990a9e03beadc0c76 (<class 'rdflib.graph.Graph'>)>

In [56]:
# authority = 'http://id.loc.gov/authorities/names/n80002329'
url = f'http://localhost:8000/authorities/import/loc/agents?authority={authority}'
r = httpx.get(url)
r.status_code

200

In [57]:
request = Agents(**r.json())
request

Agents(type='PersonalName', adminMetadata=AdminMetadata(assigner='http://id.loc.gov/vocabulary/organizations/dlc', descriptionModifier='http://id.loc.gov/vocabulary/organizations/brmninpa', creationDate=datetime.date(2023, 5, 26), descriptionLanguage='http://id.loc.gov/vocabulary/languages/por', generationProcess='BiblioKeia v.1', generationDate=datetime.datetime(2023, 5, 26, 14, 49, 50), identifiedBy=[IdentifiedBy(type='Lccn', assigner='http://id.loc.gov/vocabulary/organizations/dlc', value='n80002329')], status=Status(value='mstatus:new', label='novo')), elementList=[Element(type='FullNameElement', elementValue=Label(value='Machado de Assis,', lang=None)), Element(type='DateNameElement', elementValue=Label(value='1839-1908', lang=None))], fullerName=Element(type='PersonalName', elementValue=Label(value='Joaquim Maria Machado', lang=None)), birthDate='1839-06-21', birthPlace='Rio de Janeiro (Brazil)', deathDate='1908-09-29', hasAffiliation=[Affiliation(organization=Uri(value='http://i

In [ ]:
def getElementList(graph, authority, obj):
    
  qElementList = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?type ?value 
    WHERE  {{
    <{authority}> madsrdf:elementList ?elementList .
    ?elementList rdf:rest* ?rest .
    ?rest rdf:first ?first .
    ?first rdf:type ?type  .
    ?first madsrdf:elementValue ?value
      }}"""
  r = graph.query(qElementList)
  bindings = r.bindings
  elementList = list()

  for i in bindings:
    element = {
            "type": i.get('type').split("#")[1],
          "elementValue": {
            "value":  i.get('value').toPython(),
            "lang": i.get('lang')
          }
        }
    elementList.append(element)
  obj['elementList'] = elementList

  return obj 

def getFullerName(graph, authority, obj):
    
  qFullerName = f"""PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?type ?value 
    WHERE  {{
    <{authority}> madsrdf:fullerName ?fullerName .
      ?fullerName rdf:type ?type .
      ?fullerName rdfs:label ?value
      }}"""
  r = graph.query(qFullerName)
  bindings = r.bindings
  if len(bindings) > 0:
    result = bindings[0]
    element = {
              "type": result.get('type').split("#")[1],
            "elementValue": {
              "value": result.get('value').toPython(),
              "lang": result.get('lang')
            }
          }
    obj['fullerName'] = element
    
  return obj

def GetUri(authority, graph, metadata, obj):

    query = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
            SELECT * WHERE  {{
              <{authority}> madsrdf:{metadata} ?value .
              ?value madsrdf:authoritativeLabel ?label
              }}"""
    response = graph.query(query)
    if len(response.bindings) > 0:
        external = list()
        for i in response.bindings:
            url = i.get('value').toPython()
            base = url.split("//")[1].split("/")[0]
            uri = {
                   "value": url,
                   "base": base,
                   "label": i.get('label').value }
            external.append(uri)
        obj[metadata] = external
        
    return obj

def GetVariant(authority, graph, obj):
    
  q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
  PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
  SELECT ?typeVariant ?elementList ?elementValue ?typeElement 
  WHERE  {{      	
  <{authority}> madsrdf:hasVariant ?variant .
                  ?variant rdf:type ?typeVariant .
                  ?variant madsrdf:elementList ?elementList .
                  ?elementList rdf:rest* ?node .
                    ?node rdf:first ?e .
                    ?e madsrdf:elementValue ?elementValue .
                  ?e rdf:type ?typeElement .
                  FILTER ( ?typeVariant != madsrdf:Variant )
                  }}"""
  result = graph.query(q)
  
  if len(result.bindings) > 0:
    d = {}
    for i in result.bindings:
        bn = i.get('elementList').n3()
        typeVariant = i.get('typeVariant').split("#")[-1]
        elementValue = i.get('elementValue').toPython()
        tipo = i.get('typeElement').split("#")[-1]
        element = {'type': tipo, 'elementValue': {'value': elementValue}}
        
        if bn in d.keys():
            d[bn]['elementList'].append(element)
        else:
            d[bn] = {'type': typeVariant, 'elementList': [element]}
    variants = list(d.values())
    obj['hasVariant'] = variants
    return obj
  else:
     return obj

def GetDate(rwo, typeDate, graph, obj):

    q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?date
    WHERE  {{      	
    <{rwo}> madsrdf:{typeDate} ?date . }}"""
    result = graph.query(q)

    if len(result.bindings) > 0:
        [birthDate] = result.bindings
        obj[typeDate] = birthDate.get('date').n3().split('"')[1]
    return obj

def GetBirthPlace(rwo, graph, obj):

    q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT ?label
    WHERE  {{      	
    <{rwo}> madsrdf:birthPlace ?birthPlace .
  ?birthPlace rdfs:label ?label }}"""
    result = graph.query(q)

    if len(result.bindings) > 0:
        [birthPlace] = result.bindings
        obj['birthPlace'] = birthPlace.get('label').toPython()

    return obj

def GetAffiliation(rwo, graph, obj):

    q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?organization ?labelOrg ?start ?end
    WHERE  {{      	
    <{rwo}> madsrdf:hasAffiliation ?hasAffiliation .
  ?hasAffiliation madsrdf:organization ?organization .
  ?organization madsrdf:authoritativeLabel ?labelOrg .
  ?hasAffiliation madsrdf:affiliationStart ?start .
  OPTIONAL {{ ?hasAffiliation madsrdf:affiliationEnd ?end  }} }}"""
    result = graph.query(q)
    
    if len(result.bindings) > 0:
        affiliations = list()
        for i in result.bindings:
          # organization
          orgUri = i.get('organization').toPython()
          orgBase = orgUri.split('//')[1].split("/")[0]
          organization = {'value': orgUri, 'label': i.get('labelOrg').toPython(), 'base': orgBase}
          affiliation = {'organization': organization,
                         'affiliationStart': i.get('start').toPython(),
                         'affiliationEnd': i.get('end').toPython()}
          affiliations.append(affiliation)
        obj['hasAffiliation'] = affiliations

    return obj

def GetOccuption(rwo, graph, obj):

    q = f"""PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    SELECT ?occupation ?label
    WHERE  {{      	
    <{rwo}> madsrdf:occupation ?occupation .
   ?occupation madsrdf:authoritativeLabel ?label  }} """
    result = graph.query(q)
    if len(result.bindings) > 0:
        occupations = list()
        for i in result.bindings:
            value = i.get('occupation').toPython()
            base = value.split('//')[1].split("/")[0]
            occupation = {'value': value,
                        'label': i.get('label').value,
                        'base': base}
            occupations.append(occupation)
        obj['occupation'] = occupations
    
    return obj


token = authority.split("/")[-1]
rwo = f'http://id.loc.gov/rwo/agents/{token}'
obj = GetOccuption(rwo, graph, {})
obj

In [10]:
def ParserAgents(graph, authority):
    # Type
    tipo = GetType(graph, authority)

    # adminMetadata
    adminMetadata = {
      "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
      "identifiedBy": [ {
         "type": "Lccn",
          "assigner": "http://id.loc.gov/vocabulary/organizations/dlc",
          "value": authority.split('/')[-1]        
      }]}
    
    obj = {
     "type": tipo,
      "adminMetadata": adminMetadata,
      "isMemberOfMADSCollection": f'http://bibliokeia.com/authorities/{tipo}/'}
    
    # ElementList
    obj = getElementList(graph, authority, obj)
    
    # fullerName
    obj = getFullerName(graph, authority, obj)

    # hasCloseExternalAuthority
    obj = GetUri(authority, graph, "hasCloseExternalAuthority", obj)

    # hasExactExternalAuthority
    obj = GetUri(authority, graph, "hasExactExternalAuthority", obj)

    # Variant
    obj = GetVariant(authority, graph, obj)

    # RWO
    token = authority.split("/")[-1]
    rwo = f'http://id.loc.gov/rwo/agents/{token}'
    # BirthDate
    obj = GetDate(rwo, 'birthDate', graph, obj)
    # deathDate
    obj = GetDate(rwo, 'deathDate', graph, obj)
    # BirthPlace
    obj = GetBirthPlace(rwo, graph, obj)
    # Affiliation
    obj = GetAffiliation(rwo, graph, obj)
    # Occuptions
    obj = GetOccuption(rwo, graph, obj)
    
    return obj

obj = ParserAgents(graph, authority)
request = Agents(**obj)
request.hasAffiliation

In [37]:
obj = GetUri(authority, graph, "hasExactExternalAuthority", {})
obj

{}

In [20]:
from api.src.function.authorities.makeElement import MakeElement
from api.src.function.authorities.makeIdentifier import MakeIdentifier
from api.src.function.authorities.makeLabel import MakeLabel
from api.src.function.authorities.makeVariant import MakeVariant

id = 'TESTE'
MakeVariant(request.hasVariant)

'[ a madsrdf:PersonalName,\n                madsrdf:Variant ;\n            madsrdf:elementList ( [ a madsrdf:FullNameElement ;\n                madsrdf:elementValue "Salamage, Ruoze" ] ) ;\n            madsrdf:variantLabel "Salamage, Ruoze" ], [ a madsrdf:PersonalName,\n                madsrdf:Variant ;\n            madsrdf:elementList ( [ a madsrdf:FullNameElement ;\n                madsrdf:elementValue "Saramago, Zhoze" ] ) ;\n            madsrdf:variantLabel "Saramago, Zhoze" ], [ a madsrdf:PersonalName,\n                madsrdf:Variant ;\n            madsrdf:elementList ( [ a madsrdf:FullNameElement ;\n                madsrdf:elementValue "Сарамаго, Жозе" ] ) ;\n            madsrdf:variantLabel "Сарамаго, Жозе" ], [ a madsrdf:PersonalName,\n                madsrdf:Variant ;\n            madsrdf:elementList ( [ a madsrdf:FullNameElement ;\n                madsrdf:elementValue "סאראמאגו, ז׳וזה" ] ) ;\n            madsrdf:variantLabel "סאראמאגו, ז׳וזה" ], [ a madsrdf:PersonalName,\n  

In [52]:
request.occupation

[Uri(value='http://id.loc.gov/authorities/subjects/sh85092863', label='Novelists', base='id.loc.gov'),
 Uri(value='http://id.loc.gov/authorities/subjects/sh85039342', label='Dramatists', base='id.loc.gov'),
 Uri(value='http://id.loc.gov/authorities/subjects/sh85070788', label='Journalists', base='id.loc.gov'),
 Uri(value='http://id.loc.gov/authorities/subjects/sh2006004988', label='Nobel Prize winners', base='id.loc.gov')]

In [73]:
def MakeAffiliation(hasAffiliation):
    affiliationsList = list()
    for i in hasAffiliation:
        affiliation = f"""[ a madsrdf:Affiliation ;
           { f'madsrdf:affiliationEnd "{i.affiliationEnd}" ;' if i. affiliationEnd else '' } 
            madsrdf:affiliationStart "{i.affiliationStart}" ;
            madsrdf:organization <{i.organization.value}> ] """
        affiliationsList.append(affiliation)
    a = ", ".join(affiliationsList)
    affiliations = f"""madsrdf:hasAffiliation {a} ;"""
    return affiliations
affiliations = MakeAffiliation(request.hasAffiliation)
print(affiliations)

madsrdf:hasAffiliation [ a madsrdf:Affiliation ;
           madsrdf:affiliationEnd "1908" ; 
            madsrdf:affiliationStart "1897" ;
            madsrdf:organization <http://id.loc.gov/authorities/names/n82059239> ]  ;


In [53]:
prefix = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabulary/millus/>"""

def MakeGraphAgents(request, id):
    graph = f"""{prefix}    
    INSERT DATA {{
        GRAPH <https://bibliokeia.com/authorities/{request.type}/{id}>
        {{
        <https://bibliokeia.com/authorities/{request.type}/{id}> a madsrdf:Authority, 
            madsrdf:{request.type} ;
            madsrdf:adminMetadata [ a bf:AdminMetadata ;
            bf:assigner <{request.adminMetadata.assigner}> ;
            bf:creationDate "{request.adminMetadata.creationDate}"^^xsd:date ;
            bf:descriptionLanguage <http://id.loc.gov/vocabulary/languages/eng> ;
            bf:descriptionModifier <{request.adminMetadata.assigner}> ; 
            bf:generationProcess [ a bf:GenerationProcess ;
                    rdfs:label "{request.adminMetadata.generationProcess}" ;
                    bf:generationDate "{request.adminMetadata.generationDate}"^^xsd:dateTime ] ;
            bf:identifiedBy {MakeIdentifier(request.adminMetadata.identifiedBy, id)} ; 
            bf:status {request.adminMetadata.status.value} ] ; 
            madsrdf:authoritativeLabel "{MakeLabel(request.elementList)}" ;
            madsrdf:elementList ( {MakeElement(request.elementList)} ) ; 
            { FullerName(request) if request.fullerName else ''  }
            { f'madsrdf:birthDate "{request.birthDate}" ;' if request.birthDate else ''  }
            { BirthPlace(request) if request.birthPlace else ''  }
            { f'madsrdf:deathDate "{request.deathDate}" ;' if request.deathDate else ''  }
            { f'madsrdf:occupation {", ".join([ f"<{i.value}>" for i in request.hasCloseExternalAuthority])} ;' if request.hasCloseExternalAuthority else ''}
            { f'madsrdf:hasCloseExternalAuthority {", ".join([ f"<{i.value}>" for i in request.hasCloseExternalAuthority])} ;' if request.hasCloseExternalAuthority else ''}
            { f'madsrdf:hasExactExternalAuthority {", ".join([ f"<{i.value}>" for i in request.hasExactExternalAuthority])} ;' if request.hasExactExternalAuthority else ''}
            { f'madsrdf:hasVariant { MakeVariant(request.hasVariant) } ;' if request.hasVariant else ''  }
            madsrdf:isMemberOfMADSCollection <{request.isMemberOfMADSCollection}> .          
            }} 
        }}"""
    return graph

response = MakeGraphAgents(request, id)
print(response)

PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#> 
    PREFIX owl: <http://www.w3.org/2002/07/owl#> 
    PREFIX ri: <http://id.loc.gov/ontologies/RecordInfo#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX identifiers: <http://id.loc.gov/vocabulary/identifiers/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/> 
    PREFIX bkw: <https://bibliokeia.com/resources/work/> 
    PREFIX menclvl: <http://id.loc.gov/vocabulary/menclvl/>
    PREFIX mstatus: <http://id.loc.gov/vocabulary/mstatus/>
    PREFIX contentTypes: <http://id.loc.gov/vocabulary/contentTypes/>
    PREFIX relators: <http://id.loc.gov/vocabulary/relators/>
    PREFIX genreForms: <http://id.loc.gov/authorities/genreForms/>
    PREFIX msupplcont: <http://id.loc.gov/vocabulary/msupplcont/>
    PREFIX millus: <http://id.loc.gov/vocabul

SOLR

In [ ]:
birthDate: Optional[str]
    birthPlace: Optional[str]
    deathDate: Optional[str]
    hasAffiliation: Optional[list[Affiliation]]
    occupation: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    contributorOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str

In [131]:
def GetImagem(uri):
    id = uri.split('/')[-1]

    url = 'https://www.wikidata.org/w/api.php'
    params = {
                'action': 'wbgetentities',
                'ids': id,
                'props': 'claims',
                'languages': 'pt',
                'format': 'json'
            }
    response = httpx.get(url, params=params) 
    response = response.json()
    if response.get('error'):
        return False
    else:
        file = response['entities'][id]['claims']['P18'][0]['mainsnak']['datavalue']['value']
        imagem = f'http://commons.wikimedia.org/wiki/Special:FilePath/{file}'
        return imagem

In [137]:
request.occupation

[Uri(value='http://id.loc.gov/authorities/subjects/sh85092863', label='Novelists', base='id.loc.gov'),
 Uri(value='http://id.loc.gov/authorities/subjects/sh85103733', label='Poets', base='id.loc.gov'),
 Uri(value='http://id.loc.gov/authorities/subjects/sh85039342', label='Dramatists', base='id.loc.gov'),
 Uri(value='http://id.loc.gov/authorities/subjects/sh85034154', label='Critics', base='id.loc.gov'),
 Uri(value='http://id.loc.gov/authorities/subjects/sh85108733', label='Public officers', base='id.loc.gov')]

In [133]:
def MakeDocAgents(request, id):

    doc = {
            'id': id,
            'type': request.type,
            "creationDate": request.adminMetadata.creationDate.strftime('%Y-%m-%d'), 
            "label": f'{MakeLabel(request.elementList)}' ,
            "isMemberOfMADSCollection": request.isMemberOfMADSCollection
        }
    
    if request.fullerName:
        doc['fullerName'] = request.fullerName.elementValue.value
    if request.birthDate:
        doc['birthDate'] = request.birthDate
    if request.birthPlace:
        doc['birthPlace'] = request.birthPlace
    if request.deathDate:
        doc['deathDate'] = request.deathDate
    
    # hasAffiliation
    
    if request.hasAffiliation:
        affiliations = list()
        for i in request.hasAffiliation:
            a = {
                'id': f"{id}/hasAffiliation#{i.organization.value.split('/')[-1]}",
                'organization': i.organization.label,
                'affiliationStart': i.affiliationStart,
            }
            if i.affiliationEnd:
                a['affiliationEnd'] = i.affiliationEnd
            affiliations.append(a)
        doc['hasAffiliation'] = affiliations
    
    if request.hasVariant:
        variants = list()
        for i in request.hasVariant:
            label = [j.elementValue.value for j in i.elementList]
            label = " ".join(label)
            variants.append(label)
        doc['variant'] = variants

    # hasCloseExternalAuthority
    uris = list()
    for i in request.hasCloseExternalAuthority:
        uri = {
                'id': f"{id}/hasCloseExternalAuthority#{i.value.split('/')[-1]}",
                'uri': i.value, 
                'label': i.label, 
                'base': i.base }
        if i.base == 'www.wikidata.org':
            imagem = GetImagem(i.value)
            if imagem:
                doc['imagem'] = imagem

    uris.append(uri)
    doc['hasCloseExternalAuthority'] = uris

    return doc

doc = MakeDocAgents(request, id)

In [134]:
doc

{'id': 'TESTE',
 'type': 'PersonalName',
 'creationDate': '2023-05-26',
 'label': 'Machado de Assis,, 1839-1908',
 'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/PersonalName/',
 'fullerName': 'Joaquim Maria Machado',
 'birthDate': '1839-06-21',
 'birthPlace': 'Rio de Janeiro (Brazil)',
 'deathDate': '1908-09-29',
 'hasAffiliation': [{'id': 'TESTE/hasAffiliation#n82059239',
   'organization': 'Academia Brasileira de Letras',
   'affiliationStart': '1897',
   'affiliationEnd': '1908'}],
 'variant': ['Assis, Joaquim Maria Machado de, 1839-1908',
  'Assis, Machado de, 1839-1908',
  'De Assis, Joaquim Maria Machado, 1839-1908',
  'De Assis, Machado, 1839-1908',
  'Machado de Assis, Joaquim Maria, 1839-1908',
  'Machado de Assis, Joaquín María, 1839-1908',
  'Mashado de Assiz, Zhoakin, 1839-1908',
  'Semana Dr., 1839-1908'],
 'imagem': 'http://commons.wikimedia.org/wiki/Special:FilePath/Machado de Assis aos 57 anos.jpg',
 'hasCloseExternalAuthority': [{'id': 'TESTE/hasCloseE

In [107]:
from rdflib.plugins.sparql import prepareQuery
from SPARQLWrapper import SPARQLWrapper, JSON

In [136]:
from api.src.function.loc.graphExist import GraphExist


GraphExist('http://id.loc.gov/authorities/names/n85130797')

{'uri': {'type': 'uri',
  'value': 'https://bibliokeia.com/authorities/PersonalName/bkau-3'},
 'label': {'type': 'literal', 'value': 'Saramago, José'}}

In [129]:
file = response['entities']['Q311145']['claims']['P18'][0]['mainsnak']['datavalue']['value']
image = f'http://commons.wikimedia.org/wiki/Special:FilePath/{file}'
image

'http://commons.wikimedia.org/wiki/Special:FilePath/Machado de Assis aos 57 anos.jpg'

In [121]:
sparql.setReturnFormat(JSON)

In [122]:
results = sparql.query().convert()

In [123]:
results["results"]["bindings"]

[]

In [100]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)

In [135]:
r = solr.add([doc], commit=True)
r

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":535}}\n'

In [104]:
doc

{'id': 'TESTE',
 'type': 'PersonalName',
 'creationDate': '2023-05-26',
 'label': 'Machado de Assis,, 1839-1908',
 'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/PersonalName/',
 'fullerName': 'Joaquim Maria Machado',
 'birthDate': '1839-06-21',
 'birthPlace': 'Rio de Janeiro (Brazil)',
 'deathDate': '1908-09-29',
 'hasAffiliation': [{'id': 'TESTE/hasAffiliation#n82059239',
   'organization': 'Academia Brasileira de Letras',
   'affiliationStart': '1897',
   'affiliationEnd': '1908'}],
 'variant': ['Assis, Joaquim Maria Machado de, 1839-1908',
  'Assis, Machado de, 1839-1908',
  'De Assis, Joaquim Maria Machado, 1839-1908',
  'De Assis, Machado, 1839-1908',
  'Machado de Assis, Joaquim Maria, 1839-1908',
  'Machado de Assis, Joaquín María, 1839-1908',
  'Mashado de Assiz, Zhoakin, 1839-1908',
  'Semana Dr., 1839-1908'],
 'hasCloseExternalAuthority': [{'id': 'TESTE/hasCloseExternalAuthority#48407',
   'uri': 'http://id.worldcat.org/fast/48407',
   'label': 'Machado de As